In [1]:
!which python

/opt/anaconda3/envs/circuits/bin/python


In [2]:
%pip install torch datasets einops fancy-einsum tqdm plotly jaxtyping matplotlib
%pip install git+https://github.com/TransformerLensOrg/TransformerLens.git@dev

  Using cached fancy_einsum-0.0.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached jaxtyping-0.2.38-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached wadler_lindig-0.1.3-py3-none-any.whl.metadata (17 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached fancy_einsum-0.0.3-py3-none-any.whl (6.2 kB)
Using cached jaxtyping-0.2.38-py3-none-any.whl (56 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
Using cached wadler_lindig-0.1.3-py3-none-any.whl (20 kB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
Note: you may need to restart the kernel to use updated packages.
  Clonin

In [4]:
import torch
from transformer_lens import HookedTransformer

# Initialize the model
model_name = "gpt2-small"
model = HookedTransformer.from_pretrained(
    model_name,
    device="cuda" if torch.cuda.is_available() else "cpu",
    dtype=torch.float32
)

# Optional: Set model to evaluation mode
model.eval()

Loaded pretrained model gpt2-small into HookedTransformer


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [9]:
# Print model architecture details
print(f"Number of layers: {model.cfg.n_layers}")
print(f"Number of heads per layer: {model.cfg.n_heads}")
print(f"Hidden size (d_model): {model.cfg.d_model}")
print(f"Head dimension: {model.cfg.d_head}")
print(f"Vocabulary size: {model.cfg.d_vocab}")
print(f"Context window (max tokens): {model.cfg.n_ctx}")

Number of layers: 12
Number of heads per layer: 12
Hidden size (d_model): 768
Head dimension: 64
Vocabulary size: 50257
Context window (max tokens): 1024


In [5]:
test_prompt = "The quick brown fox jumped over the lazy dog"
print("Num tokens:", len(model.to_tokens(test_prompt)[0]))

def print_name_shape_hook_function(activation, hook):
    print(hook.name, activation.shape)

not_in_late_block_filter = lambda name: name.startswith("blocks.0.") or not name.startswith("blocks")

model.run_with_hooks(
    test_prompt,
    return_type=None,
    fwd_hooks=[(not_in_late_block_filter, print_name_shape_hook_function)],
)

Num tokens: 10
hook_embed torch.Size([1, 10, 768])
hook_pos_embed torch.Size([1, 10, 768])
blocks.0.hook_resid_pre torch.Size([1, 10, 768])
blocks.0.ln1.hook_scale torch.Size([1, 10, 1])
blocks.0.ln1.hook_normalized torch.Size([1, 10, 768])
blocks.0.ln1.hook_scale torch.Size([1, 10, 1])
blocks.0.ln1.hook_normalized torch.Size([1, 10, 768])
blocks.0.ln1.hook_scale torch.Size([1, 10, 1])
blocks.0.ln1.hook_normalized torch.Size([1, 10, 768])
blocks.0.attn.hook_q torch.Size([1, 10, 12, 64])
blocks.0.attn.hook_k torch.Size([1, 10, 12, 64])
blocks.0.attn.hook_v torch.Size([1, 10, 12, 64])
blocks.0.attn.hook_attn_scores torch.Size([1, 12, 10, 10])
blocks.0.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.0.attn.hook_z torch.Size([1, 10, 12, 64])
blocks.0.hook_attn_out torch.Size([1, 10, 768])
blocks.0.hook_resid_mid torch.Size([1, 10, 768])
blocks.0.ln2.hook_scale torch.Size([1, 10, 1])
blocks.0.ln2.hook_normalized torch.Size([1, 10, 768])
blocks.0.mlp.hook_pre torch.Size([1, 10, 3072])
b

In [10]:
test_prompt = "The quick brown fox jumped over the lazy dog"

patterns = []

def save_patterns(activation, hook):
    print(hook.name, activation.shape)
    patterns.append(activation)

pattern_filter = lambda name: "hook_pattern" in name

model.run_with_hooks(
    test_prompt,
    return_type=None,
    fwd_hooks=[(pattern_filter, save_patterns)],
)

patterns = torch.cat(patterns, dim=0) # shape (n_layers, n_heads, seq_Q, seq_K)
patterns.shape

blocks.0.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.1.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.2.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.3.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.4.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.5.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.6.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.7.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.8.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.9.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.10.attn.hook_pattern torch.Size([1, 12, 10, 10])
blocks.11.attn.hook_pattern torch.Size([1, 12, 10, 10])


torch.Size([12, 12, 10, 10])